In [1]:
states = ['o', 'a', 'b', 'c', 'd', 'e']


# Output the updated graph connectivity
graph_connectivity = dict()

for i in states:
    for j in states:
        if i == 'o' and j in {'c', 'e'}:
            graph_connectivity[i,j] = 1/2
        elif i == 'a' and j in {'d', 'e'}:
            graph_connectivity[i,j] = 1/2
        elif i == 'b' and j in {'c', 'd'}:
            graph_connectivity[i,j] = 1/2
        elif i == 'c' and not j in {i,'a'}:
            graph_connectivity[i,j] = 1/4
        elif i == 'd' and not j in {i, 'o'}:
            graph_connectivity[i,j] = 1/4
        elif i == 'e' and not j in{i, 'b'}:
            graph_connectivity[i,j] = 1/4
        else:
            graph_connectivity[i,j] = 0
{('o', j): graph_connectivity[('o', j)] for j in states}

{('o', 'o'): 0,
 ('o', 'a'): 0,
 ('o', 'b'): 0,
 ('o', 'c'): 0.5,
 ('o', 'd'): 0,
 ('o', 'e'): 0.5}

In [2]:
{('a', j): graph_connectivity[('a', j)] for j in states}

{('a', 'o'): 0,
 ('a', 'a'): 0,
 ('a', 'b'): 0,
 ('a', 'c'): 0,
 ('a', 'd'): 0.5,
 ('a', 'e'): 0.5}

In [3]:
{('b', j): graph_connectivity[('b', j)] for j in states}

{('b', 'o'): 0,
 ('b', 'a'): 0,
 ('b', 'b'): 0,
 ('b', 'c'): 0.5,
 ('b', 'd'): 0.5,
 ('b', 'e'): 0}

In [4]:
{('c', j): graph_connectivity[('c', j)] for j in states}

{('c', 'o'): 0.25,
 ('c', 'a'): 0,
 ('c', 'b'): 0.25,
 ('c', 'c'): 0,
 ('c', 'd'): 0.25,
 ('c', 'e'): 0.25}

In [5]:
{('d', j): graph_connectivity[('d', j)] for j in states}

{('d', 'o'): 0,
 ('d', 'a'): 0.25,
 ('d', 'b'): 0.25,
 ('d', 'c'): 0.25,
 ('d', 'd'): 0,
 ('d', 'e'): 0.25}

In [6]:
{('e', j): graph_connectivity[('e', j)] for j in states}

{('e', 'o'): 0.25,
 ('e', 'a'): 0.25,
 ('e', 'b'): 0,
 ('e', 'c'): 0.25,
 ('e', 'd'): 0.25,
 ('e', 'e'): 0}

In [7]:
# Corrected function to expand paths starting from 'e' and ending if 'a', 'b', or 'o' is reached
def expand_paths_from_e(current_state, graph, k, current_path=(), probability=1, all_paths=None):
    if all_paths is None:
        all_paths = []

    # Add the current state to the path
    current_path += (current_state,)

    # If we've reached an end state or the path is at its maximum length, record the path and its probability
    if current_state in ['a', 'b', 'o'] or len(current_path) > k:
        all_paths.append((current_path, probability))
        return all_paths

    # Get the next possible states
    next_states = [state for state in states if graph[(current_state, state)] > 0 and state != current_state]

    # Expand the path to each next state
    for next_state in next_states:
        # Calculate the new probability for the expanded path
        new_probability = probability * graph[(current_state, next_state)]
        # Recursively expand the path
        expand_paths_from_e(next_state, graph, k, current_path, new_probability, all_paths)

    return all_paths

# Run the function to expand all paths from 'e' up to k moves
k_moves = 11  # Replace with the desired number of steps
all_possible_paths_from_e = expand_paths_from_e('e', graph_connectivity, k_moves)

# Display all the paths and their probabilities
all_possible_paths_from_e;


In [8]:
def analyze_paths(all_paths):
    # Initialize counters for success, failure, and termination at max steps
    success_probability = 0
    failure_probability = 0
    termination_probability = 0

    for path, probability in all_paths:
        last_state = path[-1]
        # Check if the path is a success (reaching 'o')
        if last_state == 'o':
            success_probability += probability
        # Check if the path is a failure (reaching 'a' or 'b')
        elif last_state in ['a', 'b']:
            failure_probability += probability
        # Otherwise, the path terminated at the max steps without reaching 'o', 'a', or 'b'
        else:
            termination_probability += probability

    return success_probability, failure_probability, termination_probability

# Analyze the paths generated from the previous simulation
success_prob, failure_prob, termination_prob = analyze_paths(all_possible_paths_from_e)

(success_prob, failure_prob, termination_prob)


(0.3999186158180237, 0.5998372435569763, 0.000244140625)

In [2]:
def expand_paths_with_termination(current_state, graph, k, visited=None, current_path=(), probability=1, all_paths=None):
    if all_paths is None:
        all_paths = []
    if visited is None:
        visited = set()

    # Add the current state to the path and update visited states
    current_path += (current_state,)
    visited.add(current_state)

    # Check termination condition: both 'a' and 'b' visited or maximum length reached
    if ('a' in visited and 'b' in visited) or len(current_path) > k:
        all_paths.append((current_path, probability))
    else:
        # Get the next possible states
        next_states = [state for state in states if graph[(current_state, state)] > 0 and state != current_state]

        # Expand the path to each next state
        for next_state in next_states:
            # Calculate the new probability for the expanded path
            new_probability = probability * graph[(current_state, next_state)]
            # Recursively expand the path
            expand_paths_with_termination(next_state, graph, k, visited.copy(), current_path, new_probability, all_paths)

    return all_paths

# Run the function to expand all paths from 'o' up to k moves with the specified termination rule
k_moves = 12  # Adjust this value as needed for different scenarios
all_possible_paths_from_o = expand_paths_with_termination('o', graph_connectivity, k_moves)

# Function to analyze paths for "termination at max steps" or "visited both 'a' and 'b'"
def analyze_termination_paths(all_paths):
    termination_probability = 0
    visited_both_probability = 0

    for path, probability in all_paths:
        visited = set(path)
        # Check if both 'a' and 'b' have been visited
        if 'a' in visited and 'b' in visited:
            visited_both_probability += probability
        else:
            termination_probability += probability

    return visited_both_probability, termination_probability

# Analyze the collected paths
visited_both_prob, termination_prob = analyze_termination_paths(all_possible_paths_from_o)

(visited_both_prob, termination_prob)


(0.4928574562072754, 0.5071425437927246)

In [3]:
all_possible_paths_from_o

[(('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'b'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'd'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'e'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'e', 'o'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'e', 'a'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'e', 'c'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'e', 'd'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'b', 'c', 'o'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'b', 'c', 'b'),
  3.814697265625e-06),
 (('o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'o', 'c', 'b', 'c', 'd'),
  3.8146972